In [66]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
import custom_module as cm

In [67]:
# Loading the dataset
data = pd.read_csv("/Users/ahsan/Dropbox/My Mac (Alnoors-MBP-2.hub)/Downloads/combined_data.csv", parse_dates=True)

In [68]:
data["SETTLEMENTDATE"] = pd.to_datetime(data["SETTLEMENTDATE"])
data.index = data["SETTLEMENTDATE"]
data.drop(columns="SETTLEMENTDATE", inplace=True)

In [69]:
# The Real Test Data Set To Test Later #
X_test = data["2019-01-01":"2019-06-30"].copy()
X_test = X_test["RRP5MIN"]

In [6]:
# replace outliers by outlier threshold
data = cm.replace_outliers(data, 'RRP5MIN', 5)

In [7]:
data

,RRP5MIN,RESIDUAL_DEMAND
SETTLEMENTDATE,,
2010-10-20 00:00:00,1.963450,1296.50
2010-10-20 00:05:00,1.955553,1275.46
2010-10-20 00:10:00,1.946951,1268.40
2010-10-20 00:15:00,1.953207,1267.73
2010-10-20 00:20:00,1.835252,1261.69
...,...,...
2019-06-30 23:40:00,5.998145,546.95
2019-06-30 23:45:00,5.961220,560.19
2019-06-30 23:50:00,5.066510,559.87


In [8]:
# Scaling the RRP between 0 and 1 as required by the NN
scaler = MinMaxScaler()
data["RRP5MIN"] = scaler.fit_transform(data["RRP5MIN"].values.reshape(-1,1))
data["RESIDUAL_DEMAND"] = scaler.fit_transform(data["RESIDUAL_DEMAND"].values.reshape(-1,1))

In [9]:
train = data.loc["2017":].copy()

In [10]:
# include time lags of timeseries data for 3 days = 864
# We will use 3 days data to identify patterns to predict the next day

lags = 500
for i in range(1,lags+1):
    train["l_{}".format(i)] = train["RRP5MIN"].shift(i)
    
    
lags = 500
for i in range(1,lags+1):
    train["dl_{}".format(i)] = train["RESIDUAL_DEMAND"].shift(i)

In [11]:
# Drop NANS
train.dropna(inplace=True)
train.tail(5)

,RRP5MIN,RESIDUAL_DEMAND,l_1,l_2,l_3,l_4,l_5,l_6,l_7,l_8,...,dl_491,dl_492,dl_493,dl_494,dl_495,dl_496,dl_497,dl_498,dl_499,dl_500
SETTLEMENTDATE,,,,,,,,,,,,,,,,,,,,,
2019-06-30 23:40:00,0.532301,0.345898,0.535514,0.504410,0.500036,0.497298,0.517292,0.504057,0.498771,0.472195,...,0.297797,0.293034,0.289172,0.287698,0.301942,0.296936,0.293989,0.294389,0.302670,0.299656
2019-06-30 23:45:00,0.531433,0.348997,0.532301,0.535514,0.504410,0.500036,0.497298,0.517292,0.504057,0.498771,...,0.304011,0.297797,0.293034,0.289172,0.287698,0.301942,0.296936,0.293989,0.294389,0.302670
2019-06-30 23:50:00,0.510394,0.348922,0.531433,0.532301,0.535514,0.504410,0.500036,0.497298,0.517292,0.504057,...,0.306528,0.304011,0.297797,0.293034,0.289172,0.287698,0.301942,0.296936,0.293989,0.294389
2019-06-30 23:55:00,0.508886,0.345926,0.510394,0.531433,0.532301,0.535514,0.504410,0.500036,0.497298,0.517292,...,0.242003,0.306528,0.304011,0.297797,0.293034,0.289172,0.287698,0.301942,0.296936,0.293989
2019-07-01 00:00:00,0.510313,0.351366,0.508886,0.510394,0.531433,0.532301,0.535514,0.504410,0.500036,0.497298,...,0.243014,0.242003,0.306528,0.304011,0.297797,0.293034,0.289172,0.287698,0.301942,0.296936


In [12]:
# create feature and label dataframes
prelim_features = train.drop(['RRP5MIN', 'RESIDUAL_DEMAND'], axis=1)
prelim_labels = pd.DataFrame(train[['RRP5MIN']])


In [13]:
# format labels to 24 hour output range
for i in range(0, 288):
    prelim_labels['t_{}'.format(i)] = prelim_labels['RRP5MIN'].shift(-i)
prelim_labels.drop(['RRP5MIN'], axis=1, inplace=True)

# apply one-day discretization to the data
labels = prelim_labels[prelim_labels.index.minute == 0]
labels = labels[labels.index.hour == 0]
features = prelim_features[prelim_features.index.minute == 0]
features = features[features.index.hour == 0]

features_train = features[:'2018']
features_test = features['2019':'2019-06-30']
labels_train = labels[:'2018']

samples_train = len(features_train)
samples_test = len(features_test)
timesteps = 500

# convert pandas data frames to numpy ndarrays
features_train = features_train.to_numpy().reshape(samples_train, timesteps, 2)
features_test = features_test.to_numpy().reshape(samples_test, timesteps, 2)
labels_train = labels_train.to_numpy()

# check for correct data shape
features_train.shape, labels_train.shape

((728, 500, 2), (728, 288))

In [14]:
from keras.models import Model, load_model
from keras.layers.convolutional import Conv1D
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l1_l2

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import json

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [58]:
# split into training and validation data
X_train, X_valid, y_train, y_valid = train_test_split(features_train, labels_train, test_size=0.2, random_state=7)

In [74]:
###################### DESIGNING THE NN ###################
##########################################################
## 1D Convolution layer to avoid overfitting
## 3 layers of LSTM considering the complexity of the dataset

# Initialising
rnn = Sequential()
# Adding Conv1D Layer
#rnn.add(Conv1D(64, kernel_size=288, strides=288, padding='valid', input_shape=(X_train.shape[1],1)))
# Add LSTM layer 1st
rnn.add(LSTM(128, recurrent_activation='relu', return_sequences=True))
rnn.add(Dropout(0.1))
# Add LSTM layer 2nd
rnn.add(LSTM(64, recurrent_activation='relu'))
rnn.add(Dropout(0.1))
rnn.add(Dense(units=288))
rnn.compile(optimizer='adam', loss='mse')

In [ ]:
# train the model and calculate the performance on the test set
results, hist = cm.train_predict_evaluate(rnn, X_train, X_valid, y_train, y_valid, features_test,
                                       X_test.to_numpy().flatten(), X_test.index, scaler, 32, 200, 
                                       'simple_neural_network.hdf5', verbose=1)

Epoch 1/200
17/19 [=========================>....] - ETA: 3s - loss: nan

In [ ]:
f, ax = plt.subplots(figsize=(12, 6))
results.loc['2019-01-01':'2019-01-04'].plot(ax=ax);
ax.set_ylabel('Day-Ahead price in $/MWh', fontsize=14)
ax.set_xlabel('Date', fontsize=14)
ax.set_title('Short Term predictive Performance', fontsize=14);

In [ ]:
results

In [ ]:
cm.quantify_performance(results)

In [18]:
###################### DESIGNING THE NN ###################
##########################################################
## 1D Convolution layer to avoid overfitting
## 3 layers of LSTM considering the complexity of the dataset

# Initialising
rnn = Sequential()
# Adding Conv1D Layer
rnn.add(Conv1D(64, kernel_size=288, strides=288, padding='valid', input_shape=(X_train.shape[1],2)))
# Add LSTM layer 1st
rnn.add(LSTM(128, activation='tanh', return_sequences=True))
rnn.add(Dropout(0.1))
# Add LSTM layer 2nd
rnn.add(LSTM(64, activation='relu'))
rnn.add(Dropout(0.1))
rnn.add(Dense(units=288, activation='linear'))
rnn.compile(optimizer='adam', loss='mse')


In [61]:
rnn.fit(X_train, y_train, epochs=50)

Epoch 1/50
19/19 [==============================] - 0s 9ms/step - loss: 0.0072
Epoch 2/50
19/19 [==============================] - 0s 8ms/step - loss: 0.0075
Epoch 3/50
19/19 [==============================] - 0s 12ms/step - loss: 0.0078
Epoch 4/50
19/19 [==============================] - 0s 10ms/step - loss: 0.0071
Epoch 5/50
19/19 [==============================] - 0s 9ms/step - loss: 0.0071
Epoch 6/50
19/19 [==============================] - 0s 8ms/step - loss: 0.0073
Epoch 7/50
19/19 [==============================] - 0s 9ms/step - loss: 0.0072
Epoch 8/50
19/19 [==============================] - 0s 10ms/step - loss: 0.0071
Epoch 9/50
19/19 [==============================] - 0s 9ms/step - loss: 0.0071
Epoch 10/50
19/19 [==============================] - 0s 8ms/step - loss: 0.0068
Epoch 11/50
19/19 [==============================] - 0s 7ms/step - loss: 0.0070
Epoch 12/50
19/19 [==============================] - 0s 8ms/step - loss: 0.0071
Epoch 13/50
19/19 [===========================

In [63]:
pred = rnn.predict(features_test)

In [64]:
pred = scaler.inverse_transform(pred.flatten().reshape(-1, 1))

In [65]:
pred

array([[1370.9553],
       [1425.8619],
       [1448.3402],
       ...,
       [1497.0503],
       [1450.8162],
       [1385.2548]], dtype=float32)

In [ ]:
############################################################################################################

In [72]:
data.describe()

,RRP5MIN,RESIDUAL_DEMAND
count,914690.000000,914690.000000
mean,5.785942,1012.073922
std,27.676485,471.128683
min,-83.333333,-930.890000
25%,2.691540,703.860000
50%,4.130115,1024.135000
75%,6.557500,1316.990000
max,1208.333333,3341.580000


In [79]:
X = data[["RRP5MIN", "RESIDUAL_DEMAND"]].values

In [82]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [84]:
y = [x[0] for x in X_scaled]

In [85]:
y[:5]

[0.06603621935483871,
 0.0660301058064516,
 0.06602344580645161,
 0.06602828903225806,
 0.06593696903225807]

In [89]:
split = int(len(X_scaled)*0.8)
print(split)

731752


In [90]:
X_train = X_scaled[:split]
X_test = X_scaled[split:len(X_scaled)]
y_train = y[:split]
y_test = y[split:len(y)]

In [91]:
assert len(X_train) == len(y_train)
assert len(X_test) == len(y_test)

In [122]:
n = 500
Xtrain = []
ytrain = []
Xtest = []
ytest = []

for i in range(n, len(X_train)):
    Xtrain.append(X_train[i - n : i, : X_train.shape[1]])
    ytrain.append(y_train[i : i+278])
for i in range(n, len(X_test)):
    Xtest.append(X_test[i - n : i, : X_test.shape[1]])
    ytest.append(y_test[i : i+278])

In [126]:
val = np.array(ytrain[0])
val = np.c_[val, np.zeros(val.shape)]

In [131]:
Xtrain, ytrain = (np.array(Xtrain), np.array(ytrain))
Xtrain = np.reshape(Xtrain, (Xtrain.shape[0], Xtrain.shape[1], Xtrain.shape[2]))

Xtest, ytest = (np.array(Xtest), np.array(ytest))
Xtest = np.reshape(Xtest, (Xtest.shape[0], Xtest.shape[1], Xtest.shape[2]))

In [132]:
print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

(731252, 500, 2)
(731252,)
(182438, 500, 2)
(182438,)


In [137]:
###################### DESIGNING THE NN ###################
##########################################################
## 1D Convolution layer to avoid overfitting
## 3 layers of LSTM considering the complexity of the dataset

# Initialising
rnn = Sequential()
# Adding Conv1D Layer
rnn.add(Conv1D(64, kernel_size=288, strides=288, padding='valid', input_shape=(Xtrain.shape[1],2)))
# Add LSTM layer 1st
rnn.add(LSTM(128, recurrent_activation='relu', return_sequences=True))
rnn.add(Dropout(0.1))
# Add LSTM layer 2nd
rnn.add(LSTM(64, recurrent_activation='relu'))
rnn.add(Dropout(0.1))
rnn.add(Dense(units=288))
rnn.compile(optimizer='adam', loss='mse')

In [ ]:
rnn.fit(Xtrain, ytrain, validation_data=(Xtest,ytest), batch_size=16, epochs=50)